In [ ]:
# Demonstração do Modelo de Previsão de Churn de Alunos da Academia

Este notebook Jupyter tem como objetivo demonstrar a funcionalidade e o treinamento do modelo de Machine Learning utilizado na API para prever a probabilidade de **churn** (desistência) de alunos da academia.

## 1. Configuração e Importações Iniciais

Vamos importar as bibliotecas necessárias para a manipulação de dados, treinamento do modelo e salvamento.

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns

2. Definindo a Classe ChurnPredictor
Para simular o ambiente da API e garantir consistência, usaremos a mesma lógica da classe ChurnPredictor do arquivo churn_model.py. Copiaremos o código da classe para cá para que o notebook seja auto-suficiente na demonstração.

class ChurnPredictor:
    def __init__(self, model_path="churn_model.joblib"):
        self.model = None
        self.model_path = model_path
        self.features = ['frequencia_semanal', 'dias_desde_ultimo_checkin', 'duracao_media_visitas_minutos', 'tipo_plano_encoded']
        print(f"Inicializando ChurnPredictor. Caminho do modelo: {self.model_path}")
        self.load_model()

    def create_dummy_data(self):
        """
        Cria um conjunto de dados de exemplo para treinar o modelo.
        Em um cenário real, estes dados viriam do seu banco de dados.
        """
        data = {
            'frequencia_semanal': [5, 4, 1, 0.5, 3, 2, 0, 6, 1.5, 0.2, 4.5, 3.8, 0.8, 0, 5, 2.5, 1.2, 3.1, 0.1, 0.9, 2.8, 4.2, 0.6, 1.7, 3.5, 4.8, 0.3, 1.0, 2.3, 5.5],
            'dias_desde_ultimo_checkin': [1, 5, 10, 20, 3, 7, 40, 2, 15, 60, 4, 6, 25, 90, 8, 12, 18, 5, 30, 22, 9, 1, 35, 14, 7, 3, 50, 28, 11, 4],
            'duracao_media_visitas_minutos': [60, 55, 40, 25, 70, 50, 20, 65, 35, 15, 75, 68, 28, 10, 80, 48, 33, 62, 18, 22, 58, 72, 12, 30, 52, 78, 16, 42, 38, 85],
            'tipo_plano_encoded': [1, 2, 1, 1, 3, 2, 1, 3, 1, 1, 2, 3, 1, 1, 3, 2, 1, 3, 1, 1, 2, 3, 1, 2, 3, 3, 1, 2, 1, 3],
            'churn': [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0] # Churn: 1=sim, 0=não
        }
        df = pd.DataFrame(data)
        return df

    def train_model(self, X, y):
        """
        Treina o modelo de Regressão Logística.
        """
        print(" [ML] Treinando o modelo de previsão de churn...")
        self.model = LogisticRegression(solver='liblinear', random_state=42)
        self.model.fit(X, y)
        print(" [ML] Modelo treinado com sucesso!")
        self.save_model()

    def load_model(self):
        """
        Carrega um modelo treinado do disco, se existir.
        """
        if os.path.exists(self.model_path):
            print(f" [ML] Carregando modelo de churn de '{self.model_path}'...")
            self.model = joblib.load(self.model_path)
            print(" [ML] Modelo carregado.")
        else:
            print(" [ML] Modelo não encontrado. Treinando um novo modelo.")
            df = self.create_dummy_data()
            X = df[self.features]
            y = df['churn']
            self.train_model(X, y)

    def save_model(self):
        """
        Salva o modelo treinado no disco.
        """
        if self.model:
            joblib.dump(self.model, self.model_path)
            print(f" [ML] Modelo salvo em '{self.model_path}'.")

    def retrain_and_save_model(self):
        """
        Simula o retreinamento do modelo com novos dados (dummy data atualizado) e o salva.
        """
        print(" [ML] Acionando retreinamento do modelo de churn...")
        new_df = self.create_dummy_data() 
        X_new = new_df[self.features]
        y_new = new_df['churn']
        self.train_model(X_new, y_new)
        print(" [ML] Retreinamento concluído e modelo salvo.")

    def predict_churn_probability(self, features_data):
        """
        Prevê a probabilidade de churn para um dado conjunto de características.
        """
        if not self.model:
            self.load_model() 

        if not self.model: 
            print(" [ERROR] Modelo de churn não disponível para previsão.")
            return None

        input_df = pd.DataFrame([features_data], columns=self.features)
        
        probability = self.model.predict_proba(input_df)[:, 1][0]
        return probability

3. Preparação dos Dados (Dados de Exemplo)
Vamos usar a função create_dummy_data() da classe ChurnPredictor para gerar um conjunto de dados para nossa demonstração.

# Criar uma instância do preditor (isso pode carregar ou treinar o modelo se não existir)
predictor = ChurnPredictor()

# Gerar dados dummy para a demonstração
df_demo = predictor.create_dummy_data()
print("Primeiras 5 linhas dos dados de exemplo:")
print(df_demo.head())
print("\nInformações sobre os dados:")
df_demo.info()

4. Divisão dos Dados para Treinamento e Teste
Dividiremos nossos dados de exemplo em conjuntos de treinamento e teste para avaliar o desempenho do modelo.

X = df_demo[predictor.features]
y = df_demo['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nShape de X_train: {X_train.shape}")
print(f"Shape de X_test: {X_test.shape}")
print(f"Distribuição de Churn em y_train:\n{y_train.value_counts(normalize=True)}")
print(f"Distribuição de Churn em y_test:\n{y_test.value_counts(normalize=True)}")

5. Treinamento do Modelo
Agora, treinaremos o modelo usando os dados de treinamento. A classe ChurnPredictor já encapsula a lógica de treinamento.

# O modelo já foi treinado na inicialização do predictor ou carregado.
# Podemos forçar um retreinamento para a demonstração aqui se quisermos:
# predictor.retrain_and_save_model() 
# Ou simplesmente assegurar que ele está pronto:
predictor.load_model() # Garante que o modelo está carregado

print("\nModelo pronto para uso!")

6. Avaliação do Modelo
Vamos avaliar o desempenho do modelo nos dados de teste.

if predictor.model:
    y_pred = predictor.model.predict(X_test)
    y_prob = predictor.model.predict_proba(X_test)[:, 1]

    print("\n--- Relatório de Classificação ---")
    print(classification_report(y_test, y_pred))

    print(f"Acurácia: {accuracy_score(y_test, y_pred):.2f}")
    print(f"AUC-ROC: {roc_auc_score(y_test, y_prob):.2f}")

    # Plotar Curva ROC
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', label=f'Curva ROC (AUC = {roc_auc_score(y_test, y_prob):.2f})')
    plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Linha Aleatória')
    plt.xlabel('Taxa de Falsos Positivos (FPR)')
    plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
    plt.title('Curva ROC')
    plt.legend()
    plt.grid(True)
    plt.show()

else:
    print("Modelo não está disponível para avaliação.")

7. Demonstração de Previsão Individual
Vamos simular a previsão para um novo aluno, como a API faria.

# Exemplo de um aluno com alto risco de churn (baixa frequência, último check-in distante, duração curta)
aluno_alto_risco = {
    'frequencia_semanal': 0.1,
    'dias_desde_ultimo_checkin': 50,
    'duracao_media_visitas_minutos': 15,
    'tipo_plano_encoded': 1 # Mensal
}

prob_churn_alto_risco = predictor.predict_churn_probability(aluno_alto_risco)
print(f"\nProbabilidade de Churn para Aluno de ALTO RISCO: {prob_churn_alto_risco:.2f}")

# Exemplo de um aluno com baixo risco de churn (alta frequência, último check-in recente, duração longa)
aluno_baixo_risco = {
    'frequencia_semanal': 5.0,
    'dias_desde_ultimo_checkin': 2,
    'duracao_media_visitas_minutos': 70,
    'tipo_plano_encoded': 3 # Anual
}

prob_churn_baixo_risco = predictor.predict_churn_probability(aluno_baixo_risco)
print(f"Probabilidade de Churn para Aluno de BAIXO RISCO: {prob_churn_baixo_risco:.2f}")

8. Salvando o Modelo (Já feito automaticamente no treinamento)
O modelo é automaticamente salvo como churn_model.joblib na raiz do projeto após o treinamento. Isso garante que a API possa carregá-lo sem precisar retreiná-lo a cada inicialização.

print(f"Verificando se o arquivo do modelo '{predictor.model_path}' existe:")
if os.path.exists(predictor.model_path):
    print(f"O modelo '{predictor.model_path}' foi salvo com sucesso.")
else:
    print(f"ATENÇÃO: O modelo '{predictor.model_path}' NÃO foi encontrado. Verifique a execução do treinamento.")
